In [ ]:
import utils.features as uf
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

import tbtools.dev as tbdev

import utils.plotting.maps.simple_trace as st
import utils.plotting as up

In [ ]:
import tbtools.panda as tbpd

In [ ]:
uf = tbdev.reload(uf)

In [ ]:
window = '1 min'
dc = uf.delta_C_inferred(1, window=window)
c1 = C.resample(window).max()

c1 = c1[dc.index.min():dc.index.max()]

In [ ]:
dc.hist()

In [ ]:
df = pd.DataFrame({'C':c1, 'Delta C':dc})
df.plot.scatter(y='C', x='Delta C')

In [ ]:
print('0 minutes offset')
sns.boxplot(y='C', x='Delta C', data=df)

In [ ]:
df.columns

In [ ]:
sns.jointplot(y='C', x='DC+10m', data=df)

In [ ]:
print('Delta C from 10 minutes ago')
sns.boxplot(y='C', x='DC+10m', data=df)
sns.plt.figure()

In [ ]:
i = 0
pal = sns.palettes.cycle(sns.palettes.SEABORN_PALETTES['bright'])
for c in df:
    if c[:2] == 'DC':
        i+=1
        if i > 3: 
            break
        col = next(pal)
        print(col, c)
        fig = sns.pointplot(x=c, y='C', data=df, color=col, label=c, legend=True)

In [ ]:
# uf = tbdev.reload(uf)

def show_window(size, r=range(-20,25,2)):
    df2 = []
    df3 = []
    window = '{} min'.format(size)
    for offset in r:
        dc = uf.delta_C_inferred(line=1, window=window)
        s = df.C.groupby(dc.shift(offset, freq='min').astype(int))#.median()
        v = s.median()
        v.name = '{} min'.format(offset)
        c = s.count()
        c.name = v.name
        df2.append(v)
        df3.append(c)

    df2 = pd.DataFrame(df2)
    sns.plt.figure(figsize=(15,8))
    ax = sns.heatmap(df2)
    ax.set_ylabel('Delta C earlier than C, window of {}'.format(window))
    df3 = pd.DataFrame(df3)
    sns.plt.figure(figsize=(15,8))
    ax = sns.heatmap(df3, annot=True, fmt=',')
    ax.set_ylabel('Counts'.format(window))


In [ ]:
show_window(5, range(-40,40,3))

In [ ]:
show_window(10, range(-40,41,10))

In [ ]:
# uf = tbdev.reload(uf)

df2 = []
window = '10 min'
for offset in range(-20,25,2):
    dc = uf.delta_C_inferred(line=1, window=window)
    s = df.C.groupby(dc.shift(offset, freq='min').astype(int)).median()
    s.name = '{} min'.format(offset)
    df2.append(s)
    
df2 = pd.DataFrame(df2)
sns.plt.figure(figsize=(15,8))
ax = sns.heatmap(df2)
ax.set_ylabel('Delta C earlier than C, window of {}'.format(window))
# up.save_fig('w15/delta_c_prelim.png', target='week')

In [ ]:
df2 = []

for offset in range(-10,25):
    
    s = df.C.groupby(df['Delta C'].shift(offset, freq='min')).median()
    s.name = '{} min'.format(offset)
    df2.append(s)
    
df2 = pd.DataFrame(df2)

sns.plt.figure(figsize=(12,8))
ax = sns.heatmap(df2)
ax.set_ylabel('Minutes DC offset from C, window of 1 minute')
up.save_fig('w15/delta_c_prelim.png', target='week')

In [ ]:
sns.boxplot(x='Delta C', y='C', data=df)
sns.plt.figure()
sns.boxplot(x='Delta C t-5min', y='C', data=df)
sns.plt.figure()
sns.boxplot(x='Delta C t-10min', y='C', data=df)

In [ ]:


sns.plt.figure(figsize=(14,20))
for k,i in enumerate([-10, 0, 10, -5,0,5]):
    ax = sns.plt.subplot(5,3,1+k)
    tar = 'DC{:+d}m'.format(i)
    df[tar] = df['Delta C'].shift(freq='{} min'.format(i))
    sns.boxplot(x=tar, y='C', data=df, ax=ax)

In [ ]:
sns.boxplot(x='DC+5m', y='C', data=df)

In [ ]:
window = '1 min'
tp = uf.throughput(tx=14, window=window)
running = uf.line_running(tx=14, window=window)
df = pd.DataFrame({'C':C.resample(window).max(), 'throughput_14':tp, 'running':running}).fillna(0)

In [ ]:
df['2012-09-04 14:00:00':'2012-09-04 16:00:00'].plot()

In [ ]:
pd.DataFrame({'tp':tp, 'linestop':tp < 7})['2012-09-04 014:00:00':'2012-09-04 16:00:00'].plot()

In [ ]:
df['2012-09-04 06:00:00':'2012-09-04 16:00:00'].plot(figsize=(14,8))

In [ ]:
df.rolling(window=500).corr().iloc[:,1,0].plot(figsize=(14,8))

In [ ]:
df.cov()

In [ ]:
sela, selb = '2012-09-05','2012-09-05'
df[sela:selb]['throughput_14'].plot(linewidth=.5, secondary_y=True, legend=True)
df[sela:selb]['C'].plot(figsize=(14,8), linewidth=.5, legend=True)


In [ ]:
C.resample('H').max().plot()

## What is causing the zeros in C.hist?

In [ ]:
uf = tbdev.reload(uf)
C = uf.C()
C.hist(bins=C.nunique())

In [ ]:
C.hist(bins=C.nunique())

In [ ]:
C.nunique()

In [ ]:
v = C.rolling(lambda x: x.value_counts())#.reset_index(level=1)
v.name = 'histcount'
v = v.reset_index(level=1)
tbpd.rename_col(v, level_1='reinspection_count')
v = v.reset_index()
tbpd.rename_col(v, index='hour')

v = v.pivot(index='hour', columns='reinspection_count', values='histcount').fillna(0)
sns.plt.figure(figsize=(10,10))
sns.heatmap(v)

TODO make this plot for a rolling window instead of whole hours

TODO consider whether duplicates need special handling

In [ ]:
v = C.groupby(C.index.hour).value_counts()#.reset_index(level=1)
v.name = 'histcount'
v = v.reset_index(level=1)
tbpd.rename_col(v, level_1='reinspection_count')
v = v.reset_index()
tbpd.rename_col(v, index='hour')

v = v.pivot(index='hour', columns='reinspection_count', values='histcount').fillna(0)
sns.plt.figure(figsize=(10,10))
sns.heatmap(v)

# Make it with continuous time instead of per hours

# See what we can do with linear regression

Moved to nb Delta C modelling

Try with 

Y: 

    reinspection count at time T

X:

    reinspection count at time T-L
    reinspection delta for time [T-L-W:T-L]

In [ ]:
uf = tbdev.reload(uf)

In [ ]:
import utils.data as ud

In [ ]:
import tbtools.func as tbf

import utils.data as ud

In [ ]:
# uf = tbdev.reload(uf)
# x,y = uf.get_full_x_y(('5 min',), (('5 min', '2 min'), ('10 min', '2 min')))

In [ ]:
# ud.design_matrices.set('delta_c_5_min_line12.pickle', v)

In [ ]:
v = ud.design_matrices.get('5_min')

In [ ]:
y = v['C']
x = v.drop('C', axis=1)

In [ ]:
import sklearn.linear_model as sklin

In [ ]:
lr = sklin.LinearRegression()

In [ ]:
X = x[[c for c in x if '5 min' in c]]
mask = ~X.isnull().any(axis=1)
X = X[mask]
Y = y[mask]
lr.fit(X, Y)

In [ ]:
((Y - lr.predict(X))/(Y.median())).hist(bins=40)

In [ ]:
sns.plt.figure(figsize=(14,6))
Y.resample('H').max().plot(linewidth=.5, marker='.')
pd.Series(lr.predict(X), index=Y.index).resample('H').max().plot(linewidth=.5, marker='.')

In [ ]:
import tbtools.ml as tbml

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
lr.residues_

In [ ]:
sns.plt.plot(x,y, '.')
tbml.abline(lr.coef_[0], lr.intercept_)

In [ ]:
df = pd.DataFrame({'x':x, 'y':y })#, 'vals':1})

In [ ]:
import tbtools.panda as tbpd

In [ ]:
v = df.y.groupby(df.x).value_counts()
v.name = 'histcount'
v = v.reset_index(level=1)
tbpd.rename_col(v, level_1='reinspection_count')
v = v.reset_index()
tbpd.rename_col(v, x='delta_c')

v = v.pivot(index='delta_c', columns='reinspection_count', values='histcount')#.fillna(0)
sns.plt.figure(figsize=(10,10))
sns.heatmap(v)


In [ ]:
v = df.groupby().value_counts()#.reset_index(level=1)
v.name = 'histcount'
v = v.reset_index(level=1)
tbpd.rename_col(v, level_1='reinspection_count')
v = v.reset_index()
tbpd.rename_col(v, index='hour')

v = v.pivot(index='hour', columns='reinspection_count', values='histcount').fillna(0)
sns.plt.figure(figsize=(10,10))
sns.heatmap(v)

# Make it with continuous time instead of per hours

In [ ]:
sns.plt.figure(figsize=(14, 5))
a = sns.plt.gca()
a.plot(x, linewidth=0.5, alpha=0.5)
a.plot(y.values, linewidth=0.5)